In [4]:
%load_ext autoreload
%autoreload 2

from util_0804_5 import *
# from myalgorithm_0804_5 import algorithm

import pandas as pd

## 실험 내용
1. 5주문 번들에서의 최적 가중치 조합 탐색

### 모든 데이터셋 실험

In [9]:
weight_grid = []
for weight1 in [0, 0.5, 1, 1.5, 2]:
    for weight2 in [-3, -2.5, -2, -1.5, -1, -0.5, 0]:
        for weight3 in [-1, -0.5, 0, 0.5, 1, 1.5]:
            weight_grid.append((weight1, weight2, weight3))

In [10]:
len(weight_grid)

210

In [ ]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

testcase_names = []
min_costs = []
cts = []
process_times = []
cost_by_weight = []
min_init_cost_combs = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    ## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

    order_comb_possibility = [[True] * K for _ in range(K)]

    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            bundle1 = all_bundles[i]
            bundle2 = all_bundles[j]

            order_num1 = bundle1.shop_seq[0]
            order_num2 = bundle2.shop_seq[0]

            ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

            if not ip:
                order_comb_possibility[order_num1][order_num2] = False
                order_comb_possibility[order_num2][order_num1] = False

    optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

    min_init_cost = inf
    min_init_cost_bundles = []
    min_init_cost_rider_availables = []
    min_init_cost_comb = []

    weight_grid = []
    for weight1 in [0, 0.5, 1, 1.5, 2]:
        for weight2 in [-3, -2.5, -2, -1.5, -1, -0.5, 0]:
            for weight3 in [-1, -0.5, 0, 0.5, 1, 1.5]:
                weight_grid.append((weight1, weight2, weight3))

    start_time = time.time()
    temp_process_times = []
    temp_cost_by_weight = []
    
    for weight1, weight2, weight3 in weight_grid:
        temp_start_time = time.time()
        bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

        bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
        cost = sum((bundle.cost for bundle in bundles)) / K

        temp_end_time = time.time()

        temp_process_time = temp_end_time - temp_start_time
        temp_process_times.append(temp_process_time)

        weight_comb = [weight1, weight2, weight3]
        temp_cost_by_weight.append((weight_comb, cost))

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables
            min_init_cost_comb = weight_comb

    cost_by_weight.append(temp_cost_by_weight)
    min_init_cost_combs.append(min_init_cost_comb)

    used_rider_info = [(v.rider.type, len(v.shop_seq)) for v in min_init_cost_bundles]

    ct = [[0, 0] for _ in range(6)] # ct[order_c] = [walk_c, non_walk_c]

    for rider_type, order_c in used_rider_info:
        if rider_type == 'WALK':
            ct[order_c][0] += 1
        else:
            ct[order_c][1] += 1

    print(testcase_name, sum(temp_process_times), min_init_cost, min_init_cost_comb)

    cts.append(ct)
    min_costs.append(min_init_cost)
    testcase_names.append(testcase_name)
    process_times.append(temp_process_times)